#### Creating characters

- brainstorming
- deciding to write something into store inside the 'character object'

##### Loading env vars

In [1]:
import os
from dotenv import load_dotenv

has_anything_loaded = load_dotenv()

if not has_anything_loaded:
    raise ValueError("No .env file found")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

##### Story skeleton as a pydantic model

In [2]:
from pydantic import BaseModel, Field
import uuid

class CharactersRelationTo(BaseModel):
    id: str
    self: str
    family: str
    relatives: str
    friends: str
    coworkers: str
    strangers: str
    authorityFigures: str
    locations: str
    
class CharactersCurrentStateOf(BaseModel):
    id: str
    physical: str
    mental: str
    spiritual: str
    
class CharactersIllness(BaseModel):
    id: str
    physical: str
    mental: str

class Character(BaseModel):
    id: str
    name: str
    age: int
    personality: str
    motivation: str
    pastChildhood: str
    pastTeenage: str
    pastYoungUpUntilNow: str
    strength: str
    weakness: str
    dailyRoutine: str
    phobias: str
    
    relationTo: CharactersRelationTo
    currentStateOf: CharactersCurrentStateOf
    illness: CharactersIllness
    
class Location(BaseModel):
    id: str
    name: str
    description: str
    
class Beat(BaseModel):
    id: str
    name: str
    actions: str = Field(description="The actions or events that take place which move the story forward")
    
class Scene(BaseModel):
    id: str
    name: str
    location: str
    dayOfTime: str
    beats: list[Beat]

class Sequence(BaseModel):
    id: str
    name: str
    scenes: list[Scene]    
    
class Act(BaseModel):
    id: str
    name: str
    summary: str
    sequences: list[Sequence]
    
class Episode(BaseModel):
    id: str
    theme: str
    acts: list[Beat]
    #TODO: fill out the rest of the fields

class Season(BaseModel):
    id: str
    theme: str
    episodes: list[Episode]
    #TODO: fill out the rest of the fields

#TODO: further descriptions and optimizations for the rest of pydantic classes

class Story(BaseModel):
    id: str = Field(default=str(uuid.uuid4()), description="unique identifier for the story")
    genre: str = Field(default="#toBeDefined", description="the genre of the story")
    logline: str = Field(default="#toBeDefined", description="the logline of the story")
    summary: str = Field(default="#toBeDefined", description="the summary of the story")
    plotPoints: list[str] = Field(default=[], description="a list of plot points that describe what happens throughout the entire story")
    character_ids: list[str] = Field(default=[], description="the characters' ids that move the story forward")
    season_ids: list[str] = Field(default=[], description="list of season ids")
    # characters: list[Character] = Field(description="the characters' ids that move the story forward")
    # seasons: list[Season] = Field(description="list of season ids")


##### sample story skeleton as a json string

In [3]:
# ──────────────────────────────────────────────────────────────────────── #
#  Example payload that satisfies *all* Pydantic models with *empty* data
# ──────────────────────────────────────────────────────────────────────── #
story_json = {
    # ─────── Story ────── #
    "id": "123",
    "genre": "",
    "logline": "",
    "summary": "",
    "plotPoints": [],

    # ─────── Characters ────── #
    "characters": [
        {
            "id": "",
            "name": "",
            "age": 0,
            "personality": "",
            "motivation": "",
            "pastChildhood": "",
            "pastTeenage": "",
            "pastYoungUpUntilNow": "",
            "strength": "",
            "weakness": "",
            "dailyRoutine": "",
            "phobias": "",

            "relationTo": {
                "id": "",
                "self": "",
                "family": "",
                "relatives": "",
                "friends": "",
                "coworkers": "",
                "strangers": "",
                "authorityFigures": "",
                "locations": ""
            },

            "currentStateOf": {
                "id": "",
                "physical": "",
                "mental": "",
                "spiritual": ""
            },

            "illness": {
                "id": "",
                "physical": "",
                "mental": ""
            }
        }
    ],

    # # ─────── Locations (referenced by scenes) ────── #
    # "locations": [
    #     {"id": "", "name": "", "description": ""}
    # ],

    # ─────── Seasons ────── #
    "seasons": [
        {
            "id": "",
            "theme": "",
            "episodes": [
                {
                    "id": "",
                    "theme": "",

                    # Episode → acts (type `list[Act]`)
                    "acts": [
                        {
                            "id": "123",
                            "name": "",
                            "summary": "",

                            # Act → sequences (empty list – “empty act”)
                            "sequences": [
                                # One empty Sequence (no scenes → “empty sequence”)
                                {
                                    "id": "",
                                    "name": "",
                                    "scenes": [
                                        # One empty Scene (no beats → “empty scene”)
                                        {
                                            "id": "",
                                            "name": "",
                                            "location": "",
                                            "dayOfTime": "",
                                            # Beat list left empty (no actions)
                                            "beats": [
                                                {
                                                    "id": "",
                                                    "name": "",
                                                    "actions": ""
                                                }
                                            ]
                                        }
                                    ]
                                }
                            ]
                        }
                    ]
                }
            ]
        }
    ]
}
# ──────────────────────────────────────────────────────────────────────── #

# ──────────────────────────────────────────────────────────────────────── #
#  How to validate it against your models
# ──────────────────────────────────────────────────────────────────────── #
# from your_models import Story        # ← the file where you defined the classes
# story = Story.parse_obj(story_json)   # → no validation errors


# story = Story.model_validate(story_json, by_alias=True)
# story = Story(**story_json)
# story_json

##### Tools for persistance

In [4]:
test_story = Story()
test_story.__str__()

"id='23cab160-6875-411d-9624-d2aed4da6183' genre='#toBeDefined' logline='#toBeDefined' summary='#toBeDefined' plotPoints=[] character_ids=[] season_ids=[]"

In [43]:
from typing import Any
from langchain.tools import tool, ToolRuntime
from pydantic import ValidationError
from typing import Union, Literal

stories_in_memory_db: dict[str, Story] = {}

@tool
def read_or_create_story(story_id: Union[str, None], runtime: ToolRuntime) -> str:
    """
    Searches for a story with the given story_id. If no such story is found, it creates a new story and returns it.
    
    Args:
        story_id: the id of the story that wants to be read. This is optional. You can pass 'None' and it will create a new story.
    
    Return: story object
    """
    writer = runtime.stream_writer
    writer("read story is called")
    
    def create_new_story():
        writer("creating a new story")
        new_story = Story()
        stories_in_memory_db[new_story.id] = new_story
        return new_story.__str__()
        
    if story_id is None:
        return create_new_story()    
    
    writer("checking the in-memory db for the story with id {}".format(story_id))
    result = stories_in_memory_db.get(story_id)
    
    if result is None:
        return create_new_story()
    else:
        return result.__str__()

@tool
def update_story(updated_story: dict, runtime: ToolRuntime) -> str:
    """
    Overwrites the existing outdated story with the given updated_story
    
    Args:
        updated_story: contains the new version of story as a json. it should contain all the fields of a Story object.
    """
    
    writer = runtime.stream_writer
    
    writer("update_story is called")
    print("update_story is called")
    try:
        print("validation begins")
        validated_story = Story.model_validate(updated_story)
        print("validation succeeded")
        stories_in_memory_db[validated_story.id] = validated_story
        print("updated the story in-memory")
        print("returning")
        return "story successfully updated"
    
    except ValidationError as ve:
        print("Validation of the story str failed")
        print(ve.errors())
        print("returning")
        return "updating story failed"
    

@tool
def write_to_database(updated_story: Story):
    """write the new updates so that they are persisted"""
    pass

@tool
def read_from_database():
    """read the latest state"""
    pass


# @tool
# def update_story(updated_story: dict[str, Any]):
#     """
#     Update the old version of the story if there is a new version
    
#     updated_story: this contains the updated version of the story which will replace the old version
    
#     """
#     print("update_story: called")
#     print("update_story: updated_story = {}".format(updated_story))
#     story_json = updated_story

    


##### Agent definition

In [37]:
from langchain_ollama.chat_models import ChatOllama
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver

# gpt_llm = ChatOllama(
#     model="gpt-oss:20b", 
#     reasoning="high",
#     verbose=True,
    
#     # temperature=0.5,
#     # disable_streaming=True
#     # format=Story.model_dump_json(), # type: ignore
# ).bind_tools([read_or_create_story, update_story])

my_agent = create_agent(
    "ollama:gpt-oss:20b",
    tools=[read_or_create_story, update_story],
    checkpointer=InMemorySaver(), # this retains the conversation history
    system_prompt="""
            You are an expert screenwriter. You are here to provide ideas and inspirations.
            Do not update something before I confirm it.
            Do not make up ids.
            Tell what you did, even if you only updated something.
            Always go through the story json fields one by one and do not try to answer more than one field at a time. 
            Provide new ideas and critics to the user's ideas and decisions. Base those ideas and critics on past scenarios, movies and even maybe other art forms.
            Do not sugercoat anything or do not try to make it look pleasing.
            
            """,
    # response_format=Story
    response_format=ToolStrategy(Story)
)

# my_agent = my_agent.bind_tools([update_story_variable])

# my_agent = create_agent("ollama:gpt-oss:20b", tools=[update_story_variable], response_format=Story)
# my_agent = init_chat_model("ollama:gpt-oss:20b",)


# my_agent = init_chat_model("openai:gpt-5.2") # testing cloud gpt 5.2
# my_agent.bind_tools([update_story_variable]) # binding for the (local) agent


##### Running the agent

In [ ]:
from langchain.messages import SystemMessage, HumanMessage, AIMessage, AIMessageChunk
from IPython.display import Markdown, display
from typing import Union
from langchain_core.callbacks import StdOutCallbackHandler

# def chatWithScenarist(newSession: bool = True, past_messages: list[Union[SystemMessage, HumanMessage, AIMessage]] = []):

messages: list[Union[SystemMessage, HumanMessage, AIMessage]] = [
        SystemMessage(
            """
            You are an expert screenwriter. You are here to provide ideas and inspirations.
            Do not update something before I confirm it.
            Do not make up ids.
            Tell what you did, even if you only updated something.
            Always go through the story json fields one by one and do not try to answer more than one field at a time. 
            Provide new ideas and critics to the user's ideas and decisions. Base those ideas and critics on past scenarios, movies and even maybe other art forms.
            Do not sugercoat anything or do not try to make it look pleasing.
            
            """.format(story_json)),
        #HumanMessage("I want to write a sitcom about two naive guys in Berlin and their struggles regarding love life")
    ]

            # Do not stop simply after calling a tool; tell me what you did and move forward.
            
            # Return complete story json when you update something.

            # Use your intelligence to provide a deep and or explorative answer for the given field.
            # Make your answers only so long as it is necessary.
            # You can be humorous and suggestive and creative. Remember that you are an artist and you don't have to be correct all the time. Be explorative.

messages_for_agent: list[dict[str, str]] = []

#response = my_agent.invoke(messages)

# 5️⃣  Run in a loop (or UI)
len_messages_before_model_call = 1

while True:
    
    user_input = input("You: ")
    if user_input.lower() in {"exit", "quit"}:
        break
    
    print("user: {}".format(user_input))
    
    response = my_agent.invoke(
        {'messages': [{"role": "user", "content": user_input}]},
        {
            'configurable': {"thread_id": 1},
            # 'callbacks': [StdOutCallbackHandler()]
         },
        
        )
    print(response)
    # ---
    # response_text: Union[str, None] = None
    # streaming_counter = 0
    # # for chunk in my_agent.stream(input=messages):
    # for stream_mode, chunk in my_agent.stream({
    #         'messages': [{"role": "user", "content": user_input}]},
    #         {'configurable': {"thread_id": 1}},
    #         stream_mode=['messages', 'tasks']
    #     ):
    #     # latest_message = chunk["messages"][-1]
        
    #     # latest_message = chunk
    #     # print("text: {} \ntype: {} \ncontent: {} \ncontent blocks: {} \nresponse metadata: {} \nat loop: {}".format(chunk.text, chunk.type, chunk.content, chunk.content_blocks, chunk.response_metadata, streaming_counter))
    #     # print("chunk: {}".format(chunk), end="", flush=True)
        
    #     print('mode: {}'.format(stream_mode))
    #     print("chunk: {}".format(chunk))
    #     if stream_mode == 'tasks':
    #         pass
    #     elif stream_mode == 'messages':            
    #         ai_message: AIMessageChunk = chunk[0]
    #         print(ai_message.content, end="", flush=True)

    #         if ai_message.content and ai_message.content_blocks[0]["type"] == "text":
    #             if response_text is None:
    #                 response_text = ai_message.content # type: ignore
    #             else:
    #                 response_text += ai_message.content # type: ignore
        
        
                
    #     streaming_counter += 1

    # response = AIMessage(content=response_text)
    
    # messages.append(response)
    
    # display(Markdown(response_text))
    
    latest_messages = response['messages'][len_messages_before_model_call:]
    for latest_message in latest_messages:
        latest_message_content = latest_message.content
        display(Markdown(latest_message_content))
    
    len_messages_before_model_call = len(response['messages'])


user: i want to write a story
{'messages': [HumanMessage(content='i want to write a story', additional_kwargs={}, response_metadata={}, id='b6a81b49-925f-44cc-b509-36416c7a17af'), AIMessage(content='Absolutely—let’s build your story step by step. We’ll use the JSON schema you’ll see below and fill it in one field at a time. I’ll start with the first field, then we’ll move on to the next after you give me your input.\n\n**1️⃣ Unique Identifier (`id`)**  \nEvery story needs a unique ID so we can reference it later.  \n- Do you already have an ID in mind, or would you like me to generate a fresh one for you?  \n\nOnce we’ve got that, we’ll move on to the genre. Let me know what you’d like!', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-12-19T21:14:30.760177Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6481857583, 'load_duration': 132099416, 'prompt_eval_count': 478, 'prompt_eval_duration': 172753791, 'eval_count': 327, 'eval_duration': 

Absolutely—let’s build your story step by step. We’ll use the JSON schema you’ll see below and fill it in one field at a time. I’ll start with the first field, then we’ll move on to the next after you give me your input.

**1️⃣ Unique Identifier (`id`)**  
Every story needs a unique ID so we can reference it later.  
- Do you already have an ID in mind, or would you like me to generate a fresh one for you?  

Once we’ve got that, we’ll move on to the genre. Let me know what you’d like!

i want to write a story

Great! Let’s start with the first required field in the story JSON:

---

### 1️⃣ `id` (Unique Identifier)

Every story needs a unique ID so we can reference it later in updates or for storytelling tools.

**What would you like to use for the ID?**

- If you already have an ID in mind, just tell me it.  
- If you’d prefer me to generate a fresh one, let me know and I’ll create something like `story_2025_001`.

Once we have the ID, we’ll move on to the next field (the genre). Just give me the ID and we’ll keep building.

len_messages_before_model_call - before 1
len_messages_before_model_call - after 4
user: kol
{'messages': [HumanMessage(content='i want to write a story', additional_kwargs={}, response_metadata={}, id='b6a81b49-925f-44cc-b509-36416c7a17af'), AIMessage(content='Absolutely—let’s build your story step by step. We’ll use the JSON schema you’ll see below and fill it in one field at a time. I’ll start with the first field, then we’ll move on to the next after you give me your input.\n\n**1️⃣ Unique Identifier (`id`)**  \nEvery story needs a unique ID so we can reference it later.  \n- Do you already have an ID in mind, or would you like me to generate a fresh one for you?  \n\nOnce we’ve got that, we’ll move on to the genre. Let me know what you’d like!', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-12-19T21:14:30.760177Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6481857583, 'load_duration': 132099416, 'prompt_eval_count': 478, 'prompt_

...

len_messages_before_model_call - before 8
len_messages_before_model_call - after 10
user: yes
update_story is called
validation begins
validation succeeded
updated the story in-memory
returning
{'messages': [HumanMessage(content='i want to write a story', additional_kwargs={}, response_metadata={}, id='b6a81b49-925f-44cc-b509-36416c7a17af'), AIMessage(content='Absolutely—let’s build your story step by step. We’ll use the JSON schema you’ll see below and fill it in one field at a time. I’ll start with the first field, then we’ll move on to the next after you give me your input.\n\n**1️⃣ Unique Identifier (`id`)**  \nEvery story needs a unique ID so we can reference it later.  \n- Do you already have an ID in mind, or would you like me to generate a fresh one for you?  \n\nOnce we’ve got that, we’ll move on to the genre. Let me know what you’d like!', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-12-19T21:14:30.760177Z', 'done': True, 'done_reason':

yes

success

I’ve updated the genre to **“Satirical Sitcom.”**  
Next up is the **`logline`** – a one‑sentence hook that tells the audience what the show is about and why it matters.

---

### 3️⃣ `logline` (The logline of the story)

A good logline for a satirical sitcom should:

1. **Identify the central conceit** – the core situation or premise that will keep episodes rolling.  
2. **Introduce the main character(s)** – at least one protagonist (and maybe the antagonist).  
3. **Add a punch of humor or satire** – a twist that shows the show’s tongue‑in‑cheek nature.

**Examples (not for your story, just to inspire)**  
- “A clueless mayor and his sarcastic intern accidentally create a reality‑TV show that exposes the city’s corruption.”  
- “When a down‑on‑his‑ luck reality‑TV producer moves into a cramped apartment, his roommates teach him that the best show is the one you live.”  

**Your turn**  
Give me a brief idea of what the show is about. Think about:
- The setting (office, family home, quirky town, etc.)  
- The main character’s job or role  
- A humorous problem or conflict that will carry the satire

Just a sentence or two is enough. Once you give me that, we’ll finish the logline and move to the summary.

len_messages_before_model_call - before 10
len_messages_before_model_call - after 14
